The code is designed to be executed within [Docker](https://github.com/nazarb/2025_levees_DL/blob/main/Setup/Setup.md).
# Post-processing
Involves three steps:
- REMOVE SMALL OBJECTS # cv2.connectedComponentsWithStats(binary_raster, connectivity=8)
- CLOSING # cv2.morphologyEx(filtered_raster, cv2.MORPH_CLOSE, kernel)
- Skeletonize # 


In [ ]:
import numpy as np
import cv2
import os
import rasterio
from skimage import io
from skimage.morphology import thin
from scipy.ndimage import distance_transform_edt
from skimage.morphology import skeletonize

# ====== INPUT data ======
raster_r = input("Enter path to raster data: ").strip('"')
def get_filename_without_extension():
    return os.path.splitext(os.path.basename(raster_r))[0]
filename_to_detect = get_filename_without_extension()
def load_large_image(image_path):
    print(f"Loading large image from {image_path}")
    with rasterio.open(image_path) as src:
        image = src.read(1)  # Read first band
        transform = src.transform
    return image, transform

binary_raster, transform = load_large_image(raster_r)
binary_raster = (binary_raster > 0).astype(np.uint8)  # Ensure binary

# REMOVE SMALL OBJECTS 
min_area = 350
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_raster, connectivity=8)

# Keep only features above the selected min_area parameter 
filtered_raster = np.zeros_like(binary_raster, dtype=np.uint8)
for i in range(1, num_labels):  # skip background
    if stats[i, cv2.CC_STAT_AREA] >= min_area:
        filtered_raster[labels == i] = 1

# Save TFW file to preserve geographic location
def create_tfw_file(transform, tfw_path):
    print(f"Creating .tfw file at {tfw_path}")
    with open(tfw_path, 'w') as f:
        f.write(f"{transform.a}\n")
        f.write(f"{transform.b}\n")
        f.write(f"{transform.d}\n")
        f.write(f"{transform.e}\n")
        f.write(f"{transform.c}\n")
        f.write(f"{transform.f}\n")

# Output
output_dir = './model/Swin_UNETR/Aug'
os.makedirs(output_dir, exist_ok=True)
filtered_dir = os.path.join(output_dir, "Filtered")
os.makedirs(filtered_dir, exist_ok=True)

# Save
filtered_tif_path = os.path.join(filtered_dir, f'{filename_to_detect}_filtered.tif')
io.imsave(filtered_tif_path, (filtered_raster * 1).astype(np.uint8))
create_tfw_file(transform, os.path.splitext(filtered_tif_path)[0] + '.tfw')

print("Filtering complete. Small features removed and .tfw file saved.")

# CLOSING
kernel = np.ones((7,7),np.uint8)
closing_raster = cv2.morphologyEx(filtered_raster, cv2.MORPH_CLOSE, kernel)

# SAVE CLOSING results
closing_tif_path = os.path.join(filtered_dir, f'{filename_to_detect}_closing.tif')
cv2.imwrite(closing_tif_path, (closing_raster * 1).astype(np.uint8))
create_tfw_file(transform, os.path.splitext(closing_tif_path)[0] + '.tfw')


# Skeletonize
skeletonize_raster = skeletonize(closing_raster > 0, method='lee')
skeletonize_raster = (skeletonize_raster * 1).astype(np.uint8)

# Save the final results
skeletonize_raster_tif_path = os.path.join(output_dir, f'{filename_to_detect}_skeletonize.tif')
cv2.imwrite(skeletonize_raster_tif_path, skeletonize_raster)
create_tfw_file(transform, os.path.splitext(skeletonize_raster_tif_path)[0] + '.tfw')
